In [39]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text


import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.io as pio

In [2]:
from dotenv import dotenv_values

config = dotenv_values()

In [3]:
# define variables for the login
username = config['USER']
password = config['PASS']
host = config['HOST']
port = config['PORT']

In [4]:
#Create a database connection
url = f'postgresql://{username}:{password}@{host}:{port}/climate'

In [5]:
#With a connection string we can create an engine
engine = create_engine(url, echo=False)

In [24]:
with engine.begin() as conn:
    result=conn.execute(text("""
    
select *
from yearly_mean_temperature
where staid = 4
;
        """))
data = result.all()

data_mean_temp = pd.DataFrame(data) 
data_mean_temp['yearly_temp'] = data_mean_temp['yearly_temp'].astype(float)


In [25]:
data_mean_temp

,staid,yearly_temp,year
0,4,0.000000,1859
1,4,0.000000,1860
2,4,0.000000,1861
3,4,0.000000,1863
4,4,0.000000,1864
...,...,...,...
157,4,79.849315,2019
158,4,89.595628,2020
159,4,72.509589,2021
160,4,80.895890,2022


In [26]:
#Visualize Mean Teamperature over the years

fig = px.line(data_mean_temp, x='year', y='yearly_temp')
fig.show()


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   staid        162 non-null    int64  
 1   yearly_temp  162 non-null    float64
 2   year         162 non-null    int64  
dtypes: float64(1), int64(2)
memory usage: 3.9 KB


In [27]:
#Yearly temperature 3 different stations
with engine.begin() as conn:
    result=conn.execute(text("""
    
select *
from yearly_mean_temperature
where staid in (4,3,2)
;
        """))
data = result.all()

data_yearly_temp = pd.DataFrame(data) 
data_yearly_temp['yearly_temp'] = data_yearly_temp['yearly_temp'].astype(float)

In [28]:
data_yearly_temp

,staid,yearly_temp,year
0,4,0.000000,1859
1,2,0.000000,1860
2,3,0.000000,1860
3,4,0.000000,1860
4,2,0.000000,1861
...,...,...,...
445,4,72.509589,2021
446,2,67.106849,2022
447,4,80.895890,2022
448,2,16.965753,2023


In [29]:
fig = px.bar(data_yearly_temp, x='year', y='yearly_temp',
            color = 'staid')
fig.show()

In [30]:
#yearly temperature 3 different stations
fig = px.line(data_yearly_temp, x='year', y='yearly_temp', color='staid')
fig.show()
#fig.write_html("line_multi.html")

In [32]:
#Standard deviation temperature over the years in Germany

with engine.begin() as conn: 
    result= conn.execute(text("""

        SELECT 
            year,
            avg(min_temp) AS avg_min_temp,
            avg(max_temp) AS avg_max_temp
        FROM (
            SELECT 
                extract(year FROM date) AS year,
                min(tg) AS min_temp,
                max(tg) AS max_temp
            FROM 
                mean_temperature
            JOIN (
                SELECT 
                    stations.staid AS staid
                FROM 
                    countries
                JOIN 
                    stations ON countries.alpha2 = stations.cn
                WHERE 
                    countries.name = 'Germany'
            ) AS subquery
            ON mean_temperature.staid = subquery.staid
            GROUP BY year
        ) AS temperature_data
        GROUP BY year;


        """))
data= result.all()
data_temp_germany = pd.DataFrame(data)


In [33]:
data_temp_germany

,year,avg_min_temp,avg_max_temp
0,1781,-112.0000000000000000,248.0000000000000000
1,1782,-208.0000000000000000,255.0000000000000000
2,1783,-156.0000000000000000,221.0000000000000000
3,1784,-131.0000000000000000,214.0000000000000000
4,1785,-209.0000000000000000,219.0000000000000000
...,...,...,...
233,2019,-210.0000000000000000,320.0000000000000000
234,2020,-171.0000000000000000,294.0000000000000000
235,2021,-201.0000000000000000,289.0000000000000000
236,2022,-203.0000000000000000,306.0000000000000000


In [34]:
fig = px.line(data5, x='year', y=['avg_max_temp','avg_min_temp'])
fig.show()
#fig.write_html("line_multi.html")

In [36]:
#Min and max temperature in Berlin 
with engine.begin() as conn: 
    result= conn.execute(text("""


with subquery as (   
                    SELECT 
                        year,
                        month,
                        avg(min_temp) AS avg_min_temp,
                        avg(max_temp) AS avg_max_temp
                    FROM (
                        SELECT 
                            extract(year FROM date) AS year,
                            extract (month from date) as month,
                            min(tg) AS min_temp,
                            max(tg) AS max_temp
                        FROM 
                            mean_temperature
                        JOIN (
                            SELECT 
                                stations.staid AS staid
                            FROM 
                                countries
                            JOIN 
                                stations ON countries.alpha2 = stations.cn
                            WHERE 
                                countries.name = 'Germany' and staid = 41         
                        ) AS subquery
                        ON mean_temperature.staid = subquery.staid
                        GROUP BY year, month
                    ) AS temperature_data
                    where month = 1
                    GROUP BY year, month)
SELECT 
    year,
    avg_min_temp as avg_min_temp,
    avg_max_temp as avg_max_temp,
    AVG(avg_min_temp) OVER (ORDER BY year ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS rolling_av_min,
    AVG(avg_max_temp) OVER (ORDER BY year ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS rolling_av_max
FROM 
    subquery
ORDER BY 
    year; 


        """))
data= result.all()
data_temp_berlin = pd.DataFrame(data)


In [37]:
data_temp_berlin

,year,avg_min_temp,avg_max_temp,rolling_av_min,rolling_av_max
0,1876,-127.0000000000000000,39.0000000000000000,-77.0000000000000000,70.7500000000000000
1,1877,-45.0000000000000000,113.0000000000000000,-78.6000000000000000,70.2000000000000000
2,1878,-35.0000000000000000,59.0000000000000000,-87.3333333333333333,65.6666666666666667
3,1879,-101.0000000000000000,72.0000000000000000,-77.5714285714285714,67.1428571428571429
4,1880,-85.0000000000000000,68.0000000000000000,-67.8571428571428571,74.4285714285714286
...,...,...,...,...,...
143,2019,-55.0000000000000000,64.0000000000000000,-45.8571428571428571,85.0000000000000000
144,2020,-5.0000000000000000,101.0000000000000000,-33.4285714285714286,90.8571428571428571
145,2021,-52.0000000000000000,87.0000000000000000,-27.1666666666666667,100.5000000000000000
146,2022,-23.0000000000000000,117.0000000000000000,-29.0000000000000000,102.0000000000000000


In [38]:
fig = px.line(data_temp_berlin, x='year', y=['avg_min_temp','avg_max_temp','rolling_av_min', 'rolling_av_max'  ])
fig.show()
fig.write_png('temperature.png')
#fig.write_html("line_multi.html")